In [12]:
!pip install -q segmentation-models
!pip install -q -U albumentations

In [15]:
import os
import cv2
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# --- CRITICAL FIX FOR ATTRIBUTE ERROR ---
os.environ["SM_FRAMEWORK"] = "tf.keras"
if not hasattr(keras.utils, "generic_utils"):
    keras.utils.generic_utils = keras.utils

import segmentation_models as sm

# --- CONFIGURATION ---
CONFIG = {
    "IMG_SIZE": 320,      
    "BATCH_SIZE": 8,      
    "EPOCHS": 20,          
    "BACKBONE": "resnet34",   # <--- CHANGED THIS (EfficientNet link is dead)
    "LR": 0.0001,
}

# --- PATHS ---
BASE_PATH = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle"
TRAIN_IMG_PATH = os.path.join(BASE_PATH, "train_images")
TRAIN_MASK_PATH = os.path.join(BASE_PATH, "train_masks")
TEST_IMG_PATH = os.path.join(BASE_PATH, "test_images_padded") # Or "test_images"

# --- 1. DATA LOADER ---
def load_data():
    if not os.path.exists(TRAIN_IMG_PATH):
        print(f"❌ Error: Path not found: {TRAIN_IMG_PATH}")
        return [], []
    image_ids = sorted(os.listdir(TRAIN_IMG_PATH))
    X_paths = [os.path.join(TRAIN_IMG_PATH, img_id) for img_id in image_ids]
    y_paths = [os.path.join(TRAIN_MASK_PATH, img_id) for img_id in image_ids]
    return np.array(X_paths), np.array(y_paths)

X_paths, y_paths = load_data()
if len(X_paths) > 0:
    X_train, X_val, y_train, y_val = train_test_split(X_paths, y_paths, test_size=0.15, random_state=42)
else:
    X_train, X_val, y_train, y_val = [], [], [], []

# --- 2. DATA GENERATOR ---
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size, preprocess=None):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.preprocess = preprocess

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        x_out, y_out = [], []
        for i in range(len(batch_x)):
            img = cv2.imread(batch_x[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]))
            
            mask = cv2.imread(batch_y[i], cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]), interpolation=cv2.INTER_NEAREST)
            mask = (mask > 0).astype(np.float32)

            if self.preprocess:
                img = self.preprocess(img)

            x_out.append(img)
            y_out.append(mask[..., None])

        return np.array(x_out), np.array(y_out)

if len(X_train) > 0:
    # Get preprocessing for ResNet34
    preprocess_input = sm.get_preprocessing(CONFIG["BACKBONE"])
    train_gen = DataGenerator(X_train, y_train, CONFIG["BATCH_SIZE"], preprocess=preprocess_input)
    val_gen = DataGenerator(X_val, y_val, CONFIG["BATCH_SIZE"], preprocess=preprocess_input)

    # --- 3. MODEL ---
    sm.set_framework('tf.keras')
    model = sm.Unet(
        CONFIG["BACKBONE"], 
        classes=1, 
        activation='sigmoid',
        encoder_weights='imagenet' 
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG["LR"]),
        loss=sm.losses.bce_jaccard_loss,
        metrics=[sm.metrics.iou_score]
    )

    # --- 4. TRAIN ---
    print("Starting Training (ResNet34)...")
    checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_sm.keras', save_best_only=True, monitor='val_iou_score', mode='max')
    
    history = model.fit(
        train_gen, 
        validation_data=val_gen, 
        epochs=CONFIG["EPOCHS"], 
        callbacks=[checkpoint]
    )

    # --- 5. PREDICTION ---
    model.load_weights('best_model_sm.keras')
    
    def rle_encode(mask):
        pixels = mask.T.flatten()
        pixels = np.concatenate([[0], pixels, [0]])
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        return " ".join(str(x) for x in runs)

    test_files = sorted([f for f in os.listdir(TEST_IMG_PATH) if f.endswith(".png")])
    print(f"Generating predictions for {len(test_files)} images...")

    with open("submission.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["image_id", "encoded_pixels"])

        for filename in test_files:
            img_path = os.path.join(TEST_IMG_PATH, filename)
            
            # READ ORIGINAL DIMENSIONS
            original_img = cv2.imread(img_path)
            orig_h, orig_w = original_img.shape[:2]
            
            # Preprocess
            img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]))
            img = preprocess_input(img)
            
            # Predict
            prob = model.predict(img[None], verbose=0)[0, :, :, 0]
            
            # RESIZE TO ORIGINAL
            prob_full = cv2.resize(prob, (orig_w, orig_h))
            
            # Encode
            mask = (prob_full > 0.5).astype(np.uint8)
            encoded = rle_encode(mask)
            
            try:
                clean_id = int(filename.replace(".png", ""))
            except:
                clean_id = filename
                
            if not encoded: encoded = " "
            writer.writerow([clean_id, encoded])

    print("DONE. Submission file created.")
else:
    print("Skipping training due to missing data.")

85521592/85521592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Starting Training (ResNet34)...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


2026-02-08 15:46:55.910614: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:46:56.186292: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:46:56.436291: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng4{k11=1} for conv %cudnn-conv-bw-input.50 = (f32[8,32,160,160]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,32,160,160]{3,2,1,0} %bitcast.53647, f32[32,32,3,3]{3,2,1,0} %bitcast.39975), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", metadata={op_type="Conv2DBackpropInput" op_name="gradient_tape/functional_4_1/decoder_stage3b_conv_1/convolution/Conv2DBackpropInput" s

310/338 ━━━━━━━━━━━━━━━━━━━━ 6s 248ms/step - iou_score: 0.7372 - loss: 0.5988

2026-02-08 15:48:43.582926: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:48:43.848733: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:48:44.482326: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:48:44.768595: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-08 15:48:45.001070: E external/local_xla/xla/stream_

338/338 ━━━━━━━━━━━━━━━━━━━━ 185s 381ms/step - iou_score: 0.7446 - loss: 0.5818 - val_iou_score: 0.9119 - val_loss: 0.2042
Epoch 2/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 269ms/step - iou_score: 0.9153 - loss: 0.2030 - val_iou_score: 0.9445 - val_loss: 0.1377
Epoch 3/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 269ms/step - iou_score: 0.9414 - loss: 0.1471 - val_iou_score: 0.9544 - val_loss: 0.1175
Epoch 4/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 268ms/step - iou_score: 0.9514 - loss: 0.1255 - val_iou_score: 0.9612 - val_loss: 0.1020
Epoch 5/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 269ms/step - iou_score: 0.9565 - loss: 0.1138 - val_iou_score: 0.9641 - val_loss: 0.0965
Epoch 6/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 269ms/step - iou_score: 0.9594 - loss: 0.1074 - val_iou_score: 0.9657 - val_loss: 0.0934
Epoch 7/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 269ms/step - iou_score: 0.9609 - loss: 0.1041 - val_iou_score: 0.9666 - val_loss: 0.0901
Epoch 8/20
338/338 ━━━━━━━━━━━━━━━━━━━━ 91s 268ms/step - iou_score: 0.9610 - lo